<a href="https://colab.research.google.com/github/ZManani/Multi-class-sentiment-analysis/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
sns.set(color_codes=True)

import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords') 
#nltk.download('movie_reviews') 
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [61]:
import csv
import urllib.request as urllib2
import matplotlib.pyplot as plt
from nltk import FreqDist
import pandas as pd
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
response = urllib2.urlopen(url)
df = pd.read_csv(response,delimiter='\t',encoding='utf-8')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [62]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,129595,6968,", fits the bill perfectly",4
1,109613,5807,a praiseworthy attempt to generate suspense ra...,4
2,9182,383,of talented thesps,2
3,6764,270,have rarely,2
4,102203,5378,see this delightful comedy,4


In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=2003)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())


for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

[['decent', 'performances'], 3]

In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True

In [66]:
for l in range(len(documents)):
  label = documents[l][1]
  tmpReview = []
  for w in documents[l][0]:
    newWord = w
    if remove_stopwords and (w in stopwords_en):
      continue
    if removePuncs and (w in punctuations): 
      continue
    if useStemming:
      newWord = lancaster.stem(newWord)
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord)
    tmpReview.append(newWord)
  documents[l] = (tmpReview, label)
  documents[l] = (' '.join(tmpReview), label) 

print(documents[0])
    

('decent performances', 3)


In [67]:
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head()

,text,sentiment
0,decent performances,3
1,believe,2
2,spectacle pacing,2
3,rhapsodic dialogue jumps page memorable charac...,4
4,entry,2


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 3000,ngram_range=(1,1)) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test

In [70]:
Y_test

13510     3
61932     3
82549     1
137718    3
121990    1
         ..
94224     3
135456    2
154729    3
23031     2
57870     1
Name: sentiment, Length: 46818, dtype: int64

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [73]:
batch_size = 64
num_classes = 5
epochs = 5
X_train.shape

(109242, 3000)

In [74]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)
Y_test

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [0]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5,
                 activation='relu',
                 input_shape=(3000,1)))
model.add(Conv1D(64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [76]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy',f1_m,precision_m, recall_m])
X_test.shape[1]

3000

In [77]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
hh= model.fit(X_train, Y_train,
          batch_size=64,
          epochs=30)
model.save('/content/drive/My Drive/1105524_1dconv_reg.h5')
test_loss, test_accuracy, test_f1_score, test_precision, test_recall = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)
print('Test f1 score:', test_f1_score)
print('Test precision:', test_precision)
print('Test recall:', test_recall)

Epoch 1/30
109242/109242 [==============================] - 26s 238us/step - loss: 1.1145 - acc: 0.5630 - f1_m: 0.4892 - precision_m: 0.6112 - recall_m: 0.4152
Epoch 2/30
109242/109242 [==============================] - 25s 230us/step - loss: 1.0079 - acc: 0.6070 - f1_m: 0.5713 - precision_m: 0.6656 - recall_m: 0.5020
Epoch 3/30
109242/109242 [==============================] - 25s 230us/step - loss: 0.9817 - acc: 0.6196 - f1_m: 0.5880 - precision_m: 0.6741 - recall_m: 0.5228
Epoch 4/30
109242/109242 [==============================] - 25s 230us/step - loss: 0.9679 - acc: 0.6248 - f1_m: 0.5955 - precision_m: 0.6777 - recall_m: 0.5326
Epoch 5/30
109242/109242 [==============================] - 25s 230us/step - loss: 0.9560 - acc: 0.6310 - f1_m: 0.6025 - precision_m: 0.6806 - recall_m: 0.5417
Epoch 6/30
109242/109242 [==============================] - 25s 230us/step - loss: 0.9469 - acc: 0.6339 - f1_m: 0.6089 - precision_m: 0.6836 - recall_m: 0.5500
Epoch 7/30
109242/109242 [==============

In [78]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/1105524_1dconv_reg.h5', 
                   custom_objects = {'f1_m': f1_m,  'precision_m': precision_m, 'recall_m' : recall_m})

test_loss, test_accuracy, test_f1_score, test_precision, test_recall = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)
print('Test f1 score:', test_f1_score)
print('Test precision:', test_precision)
print('Test recall:', test_recall)

Test loss: 0.9997733558042631
Test accuracy: 0.62320047844846
Test f1 score: 0.6037681600667517
Test precision: 0.6616703046932088
Test recall: 0.5566662394805417
